# Extract text transcripts of these Youtube interviews

## Fetching the url of videos from docx file

In [ ]:
import docx
sitelist=[]
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
        print(para.text.split()[1])
        sitelist.append(para.text.split()[1])
    return len(doc.paragraphs)
        
num_vid=getText("youtube_list.docx") #youtube_list is in the home folder
#print(sitelist,num_vid)

## Downloading video in audio (.wav) format and extracting the transcripts of videos 

In [ ]:
from __future__ import unicode_literals
import youtube_dl
from os import path
from pydub import AudioSegment
import wave
import speech_recognition as sr
import sys
import contextlib
doclist=[]
for i in range (num_vid):
    ydl_opts = {
    #'format': 'bestaudio/best',
    'outtmpl': 'audfile'+str(i),
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    }
    if(not path.exists("audfile"+str(i)+".wav")):
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([sitelist[i]])
    #print("downloaded in audio format")
    
    filename="audfile"+str(i)+".wav"
    with contextlib.closing(wave.open(filename,'r')) as f:
        print(type(f))

        # files 
        if(type(f)=="mkv") :     
            try:
                src = "audfile"+str(i)+".wav"
                dst = "audfile"+str(i)+".wav"

        # convert mkv to wav
        
                sound = AudioSegment.from_mkv(src)
                sound.export(dst, format="wav")
            except:
                print("not converted to .wav")
    #transcribing
    
    r=sr.Recognizer()
    try:
        with contextlib.closing(wave.open(filename,'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            totduration = frames / float(rate)
            print(totduration)
    except:
        print("error_occured")
        totduration=1
    off_time=0
    doc=str("")
    while(totduration>11):
        with sr.AudioFile(filename) as source:
            audio=r.record(source, duration=10,offset=off_time)
        #print("bvlfsk.bsdkvdf")
        try:
            x=r.recognize_google(audio)
        except:
            print("-----this line is skipped-----")
            x=' '
        doc=doc+x+' ';
        #print(str(offtime)+":::System predicts:"+x)
        totduration-=10
        off_time+=10
    doclist.append(doc)
    #doclist is a list of all transcripts

## Cleaning the transcripts

In [ ]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doclist]  
for j in range(len(doc_clean)):
    doc_clean[j] = [word for word in doc_clean[j] if (word.isalpha() and len(word)>3) ]
    

In [ ]:
import gensim
from gensim import corpora

## Preparing a dictionary of corpus

In [ ]:
dictionary = corpora.Dictionary(doc_clean)

## Converting list of documents (corpus) into Document Term Matrix using dictionary

In [ ]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

## Using LDA model for topic modelling

In [ ]:
import numpy as np
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=50,dtype=np.float64)
for i in range(len(doc_term_matrix)):
    print ('\nTopic for video no. :'+str(i))
    templist=(ldamodel.print_topics(num_topics=20)[ldamodel[doc_term_matrix[i]][0][0]][1]).split("\"")
    #print(type(templist))
    for k in range(len(templist)):
        if(templist[k].isalpha()):
            print(templist[k],end=' ')
#this prints the topic for each of the video